In [1]:
from pfd_toolkit import load_reports, LLM, Extractor, Screener
from dotenv import load_dotenv
import os

reports = load_reports(
    refresh=True,
    end_date="2025-06-01")

print(f"{len(reports)} PFD reports were published before 1st June 2025.")

5728 PFD reports were published before 1st June 2025.


In [ ]:
# Load OpenAI API key
load_dotenv("../api.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialise LLM client
llm_client = LLM(api_key=openai_api_key, 
                 model="gpt-4.1-mini",
                 max_workers=25,
                 seed=123, 
                 timeout=30
                 )

In [ ]:
# Initalise Screener
surgical_screener = Screener(llm=llm_client,
                             reports=reports)

# Set user query & run Screener
query="Reports mentioning death following **cosmetic** procedures (both surgical and non-surgical and including breast implants)"

filtered_reports = surgical_screener.screen_reports(user_query=query,
                                 produce_spans=True)


print(f"From our initial {len(reports)} PFD reports, PFD Toolkit identified {len(filtered_reports)} matching reports.")

Sending requests to the LLM: 100%|██████████| 5728/5728 [02:34<00:00, 37.07it/s]


From our initial 5728 PFD reports, PFD Toolkit identified 7 matching reports.


In [4]:
filtered_reports

,url,id,date,coroner,area,receiver,investigation,circumstances,concerns,spans_matches_query
180,https://www.judiciary.uk/prevention-of-future-...,2025-0116,2025-02-10,I. Thistlethwaite,Rutland and North Leicestershire,The Secretary of State for Health and Social Care,On 31st May 2024 I commenced an investigation ...,Mrs Towlson was a 58 year old female who was f...,I am concerned about the fact that the inquest...,"""cosmetic surgery""; ""tummy tuck and liposuctio..."
517,https://www.judiciary.uk/prevention-of-future-...,2024-0436,2024-08-08,K. Hayes,Surrey,Secretary of State for Health & Social Care; P...,"On 15 June 2023, I commenced an investigation ...","Mrs Stokes died of radiation induced sarcoma, ...",I am concerned that there is not any or insuff...,"""breast implant""; ""reconstructive surgery whic..."
1156,https://www.judiciary.uk/prevention-of-future-...,2023-0330,2023-09-13,J. Lake,Norfolk,The Secretary of State for Health and Social C...,On 9 December 2019 I commenced an investigatio...,Melissa Kerr was admitted to Private Medicana ...,Expert evidence was heard that the findings in...,"""underwent surgery to harvest fat from the abd..."
2276,https://www.judiciary.uk/prevention-of-future-...,2021-0064,2021-03-04,M. Hassell,Inner North London,Chief Executive Phoenix Hospital Group,"On 6 October 2020, I commenced an investigatio...",Paula Speirs died from the ingestion of benzod...,When Ms Speirs arrived at Weymouth Street Hosp...,"""submental recontouring and a mini face lift"";..."
2627,https://www.judiciary.uk/prevention-of-future-...,2020-0260,2020-01-31,J. Bennett,Birmingham and Solihull,British Association of Aesthetic & Plastic Sur...,On 10th September 2019 I commenced an investig...,Mrs Brooks had debilitating lipoedema from the...,Large volume liposuction is the recognised tre...,"""liposuction procedures""; ""Large volume liposu..."
2715,https://www.judiciary.uk/prevention-of-future-...,2019-0408,2019-11-29,K. McLoughlin,West Yorkshire,"Joseph Hill, Solicitors; The Secretary of Stat...",On 5 September 2018 an investigation was comme...,Leah L Cambridge aged 29 travelled to Izmir Tu...,Notwithstanding the death of Ms Cambridge on 2...,"""cosmetic surgical procedure""; ""Brazilian Butt..."
4401,https://www.judiciary.uk/prevention-of-future-...,2016-0113,2016-03-18,M. Hassell,Inner North London,Medical Director Homerton University Hospital ...,"On 25 November 2015, one of my assistant coron...",Ms Pathan was admitted to Homerton University ...,Although Ms Pathan's MENTOR breast implant was...,"""breast reconstruction wound""; ""Ms Pathan's ME..."


In [12]:
from pydantic import BaseModel, Field

llm_client.model="gpt-4.1"

# Define feature model with pydantic
class MyFeatures(BaseModel):
    name: str = Field(description="The name of the deceased, where provided.")
    age: int = Field(description="The age of the deceased, where provided.")
    procedure: str = Field(description="The names of the cosmetic procedure(s) the deceased received, as stated in the report.")
    cause_of_death: str = Field(description="Specific cause of death following the procedure, where provided.")
    consent: str = Field(description="What the report says about the effectiveness of informed consent, if anything. Verbatim.")
    product_regulation: str = Field(description="What the report says about Product and provider regulation, if anything. Verbatim.")
    public_education: str = Field(description="What the report says about public education, if anything. Verbatim.")
    research_gaps: str = Field(description="What the reports says about Research gaps, if anything. Verbatim.")
    recourse: str = Field(description="What the report says about Remedy and recourse, if anything. Verbatim.")
    cosmetic_tourism: str = Field(description="What the report says about Cosmetic tourism, if anything. Verbatim.")
    

extractor = Extractor(llm=llm_client, reports=filtered_reports)

# Extract features
reports_features = extractor.extract_features(feature_model=MyFeatures,
                           produce_spans=True,
                           drop_spans=True,
                           allow_multiple=True)

Extracting features: 100%|██████████| 7/7 [00:17<00:00,  2.55s/it]


In [13]:
# Add a summary column

reports_features = extractor.summarise()

Summarising reports:   0%|          | 0/7 [00:00<?, ?it/s]

In [14]:
reports_features.to_csv("../data/cosmetic_surgery.csv")